Visualization utilities for menu item recommendations and popularity scores.

This module provides functions for plotting interactive bar charts to visualize the popularity of recommended menu items using Altair.
It includes tools for formatting, color-coding, and exporting visualizations to HTML

In [1]:
import altair as alt
import pandas as pd
import json
from menu_recommender import recommend_menu_items

def plot_popularity_chart(json_str: str, width: int = 800, height: int = 600, export_html: bool = False):
    """
    Generates and displays (or exports) a bar chart visualizing the popularity of menu items.

    Args:
        json_str (str): JSON-formatted string representing a list of menu item records.
            Each record must contain the fields: 'dish_base', 'dish_flavor', and 'popularity_score'.
        width (int, optional): Width of the chart in pixels. Defaults to 800.
        height (int, optional): Height of the chart in pixels. Defaults to 600.
        export_html (bool, optional): If True, returns the chart as an HTML string; otherwise, displays the chart in the browser. Defaults to False.

    Returns:
        None or str: If export_html is True, returns the chart as an HTML string. Otherwise, displays the chart and returns None.

    Example:
        records = [
            {"dish_base": "pizza", "dish_flavor": "cheese", "popularity_score": 0.91},
            {"dish_base": "pasta", "dish_flavor": "alfredo", "popularity_score": 0.87}
        ]
        plot_popularity_chart(json.dumps(records), width=600, height=400)

    Notes:
        - The top three dishes are color-coded for emphasis.
        - Popularity scores are displayed as percentages.
        - Rank labels ('Top 1', 'Top 2', 'Top 3') are annotated for the highest-ranking dishes.
        - Requires Altair and pandas for visualization and data processing.
    """
    # Custom colors
    heymate_yello = '#F5D866'
    heymate_red = '#ED5751'
    rank_3_color = '#df979e'
    rank_other = '#777777'

    # Parse the JSON string
    records = json.loads(json_str)

    # Create DataFrame and combine dish_base and dish_flavor
    df = pd.DataFrame(records)
    df['Dish Name'] = df.apply(
        lambda row: f"{row['dish_base'].title()} ({row['dish_flavor'].title()})", axis=1)
    df['Popularity Score'] = df['popularity_score'] * 100

    # Sort the DataFrame
    df = df.sort_values('Popularity Score', ascending=False).reset_index(drop=True)
    num_items = len(df)

    # Generate color list
    chart_colors = [heymate_yello, heymate_red, rank_3_color] + [rank_other] * max(0, num_items - 3)
    chart_colors = chart_colors[:num_items]

    # Generate rank labels
    chart_labels = ['Top 1', 'Top 2', 'Top 3'] + [''] * max(0, num_items - 3)
    chart_labels = chart_labels[:num_items]

    # Assign to DataFrame
    df['Color'] = chart_colors
    df['Rank Label'] = chart_labels
    sorted_dish_names = df['Dish Name'].tolist()

    # Bar chart
    bars = alt.Chart(df).mark_bar(size=35).encode(
        x=alt.X('Popularity Score:Q',
                scale=alt.Scale(domain=[0, 100]),
                axis=alt.Axis(title="Popularity Score", values=[50, 75, 100])),
        y=alt.Y('Dish Name:N',
                sort=sorted_dish_names,
                axis=alt.Axis(title="Dish Name", labelAngle=-5),
                scale=alt.Scale(paddingInner=1/2)),
        color=alt.Color('Color:N', scale=None, legend=None),
        tooltip=[
            alt.Tooltip('Dish Name', title='Dish'),
            alt.Tooltip('Popularity Score', title='Score', format=".1f")
        ]
    ).properties(
        title='Top Dish Popularity Scores',
        width=width,
        height=height
    )

    # Rank text
    rank_text_labels = alt.Chart(df[df['Rank Label'] != '']).mark_text(
        align='left',
        baseline='middle',
        dx=20,
        fontSize=18,
        fontWeight='bold'
    ).encode(
        x='Popularity Score:Q',
        y=alt.Y('Dish Name:N', sort=sorted_dish_names),
        text='Rank Label:N',
        color=alt.Color('Color:N', scale=None, legend=None)
    )

    # Score labels inside bars
    score_text_labels = alt.Chart(df).mark_text(
        align='right',
        baseline='middle',
        dx=-5,
        fontSize=12,
        color='white'
    ).encode(
        x=alt.X('Popularity Score:Q'),
        y=alt.Y('Dish Name:N', sort=sorted_dish_names),
        text=alt.Text('Popularity Score:Q', format=".1f")
    )

    # Final chart
    chart = (bars + rank_text_labels + score_text_labels).configure_axis(
        labelFontSize=14,
        titleFontSize=16,
        grid=False
    ).configure_title(
        fontSize=22,
        anchor='start'
    ).configure_view(
        fill=None,
        strokeWidth=0
    )

    if export_html:
        return chart.to_html()
    else:
        chart.show()

In [2]:
# Example
recommendation_json = json.dumps(recommend_menu_items(type1="pizza restaurant", type2="burger joint", type3="sandwich shop"))
plot_popularity_chart(recommendation_json)

/Users/krabssiew/Desktop/MDS/Block 6/capstone/heymate-report/script/sql_reader.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Read 24223 rows from SQL Server.


alt.LayerChart(...)